# Creation of json file for sentinel-1-rtc

In [1]:
import os
from pathlib import Path
import json
import rasterio
from datetime import datetime

In [2]:
def get_image_metadata(tiff_path):
    with rasterio.open(tiff_path) as src:
        bbox = list(src.bounds)
        transform = list(src.transform)
        epsg = src.crs.to_epsg()
        shape = list(src.shape)

        centroid = {
            "lat": (bbox[1] + bbox[3]) / 2,
            "lon": (bbox[0] + bbox[2]) / 2
        }

    return {
        "bbox": bbox,
        "proj:epsg": epsg,
        "proj:transform": transform,
        "proj:shape": shape,
        "proj:centroid": centroid
    }

In [29]:
def create_s1_rtc_item_json(scene_id, scene_dir, base_url):
    index_files = list(scene_dir.glob("*.tif"))
    if not index_files:
        print(f"No TIFF files found for {scene_id}")
        return None

    metadata = get_image_metadata(index_files[0])

    try:
        date_str = next(f.stem.split('_')[-1] for f in index_files if 'sentinel1' in f.stem)
        scene_datetime = datetime.strptime(date_str, "%Y-%m-%d")
    except (IndexError, ValueError):
        print(f"Error parsing datetime for {scene_id}")
        return None

    item = {
        "type": "Feature",
        "stac_version": "1.0.0",
        "stac_extensions": [
            "https://stac-extensions.github.io/sar/v1.0.0/schema.json",
            "https://stac-extensions.github.io/sat/v1.0.0/schema.json",
            "https://stac-extensions.github.io/item-assets/v1.0.0/schema.json",
            "https://stac-extensions.github.io/table/v1.2.0/schema.json"
        ],
        "id": scene_id,
        "collection": "sentinel-1-rtc",
        "bbox": metadata["bbox"],
        "geometry": {
            "type": "Polygon",
            "coordinates": [[
                [metadata["bbox"][0], metadata["bbox"][1]],
                [metadata["bbox"][0], metadata["bbox"][3]],
                [metadata["bbox"][2], metadata["bbox"][3]],
                [metadata["bbox"][2], metadata["bbox"][1]],
                [metadata["bbox"][0], metadata["bbox"][1]]
            ]]
        },
        "properties": {
            "datetime": scene_datetime.strftime("%Y-%m-%dT%H:%M:%SZ"),
            "platform": "sentinel-1",
            "instruments": ["c-sar"],
            "sentinel:product_type": "rtc",
            **metadata
        },
        
        "links": [
            {
                "rel": "root",
                "href": f"{base_url}/collection.json",
                "type": "application/json"
            },
            {
                "rel": "parent",
                "href": f"{base_url}/collection.json",
                "type": "application/json"
            },
            {
                "rel": "collection",
                "href": f"{base_url}/collection.json",
                "type": "application/json"
            }
        ],
        
        "assets": {
            index_file.stem: {
                "href": f"{base_url}/{scene_id}/{index_file.name}",
                "type": "image/tiff; application=geotiff; profile=cloud-optimized",
                "title": f"Sentinel-1 RTC {index_file.stem}",
                "description": f"Sentinel-1 RTC {index_file.stem} polarization",
                "roles": ["data"]
            } for index_file in index_files
        }
    }

    return item

In [33]:
def create_s1_rtc_collection_json(output_dir, items, base_url):
    all_bboxes = [item["bbox"] for item in items if item]
    overall_bbox = [
        min(b[0] for b in all_bboxes),
        min(b[1] for b in all_bboxes),
        max(b[2] for b in all_bboxes),
        max(b[3] for b in all_bboxes)
    ]

    collection = {
        "type": "Collection",
        "id": "sentinel-1-rtc",
        "stac_version": "1.0.0",
        "description": "The [Sentinel-1](https://sentinel.esa.int/web/sentinel/missions/sentinel-1) mission is a constellation of two polar-orbiting satellites, operating day and night performing C-band synthetic aperture radar imaging. The Sentinel-1 Radiometrically Terrain Corrected (RTC) data in this collection is a radiometrically terrain corrected product derived from the [Ground Range Detected (GRD) Level-1](https://planetarycomputer.microsoft.com/dataset/sentinel-1-grd) products produced by the European Space Agency. The RTC processing is performed by [Catalyst](https://catalyst.earth/).\n\nRadiometric Terrain Correction accounts for terrain variations that affect both the position of a given point on the Earth's surface and the brightness of the radar return, as expressed in radar geometry. Without treatment, the hill-slope modulations of the radiometry threaten to overwhelm weaker thematic land cover-induced backscatter differences. Additionally, comparison of backscatter from multiple satellites, modes, or tracks loses meaning.\n\nA Planetary Computer account is required to retrieve SAS tokens to read the RTC data. See the [documentation](http://planetarycomputer.microsoft.com/docs/concepts/sas/#when-an-account-is-needed) for more information.\n\n### Methodology\n\nThe Sentinel-1 GRD product is converted to calibrated intensity using the conversion algorithm described in the ESA technical note ESA-EOPG-CSCOP-TN-0002, [Radiometric Calibration of S-1 Level-1 Products Generated by the S-1 IPF](https://ai4edatasetspublicassets.blob.core.windows.net/assets/pdfs/sentinel-1/S1-Radiometric-Calibration-V1.0.pdf). The flat earth calibration values for gamma correction (i.e. perpendicular to the radar line of sight) are extracted from the GRD metadata. The calibration coefficients are applied as a two-dimensional correction in range (by sample number) and azimuth (by time). All available polarizations are calibrated and written as separate layers of a single file. The calibrated SAR output is reprojected to nominal map orientation with north at the top and west to the left.\n\nThe data is then radiometrically terrain corrected using PlanetDEM as the elevation source. The correction algorithm is nominally based upon D. Small, [“Flattening Gamma: Radiometric Terrain Correction for SAR Imagery”](https://ai4edatasetspublicassets.blob.core.windows.net/assets/pdfs/sentinel-1/2011_Flattening_Gamma.pdf), IEEE Transactions on Geoscience and Remote Sensing, Vol 49, No 8., August 2011, pp 3081-3093. For each image scan line, the digital elevation model is interpolated to determine the elevation corresponding to the position associated with the known near slant range distance and arc length for each input pixel. The elevations at the four corners of each pixel are estimated using bilinear resampling. The four elevations are divided into two triangular facets and reprojected onto the plane perpendicular to the radar line of sight to provide an estimate of the area illuminated by the radar for each earth flattened pixel. The uncalibrated sum at each earth flattened pixel is normalized by dividing by the flat earth surface area. The adjustment for gamma intensity is given by dividing the normalized result by the cosine of the incident angle. Pixels which are not illuminated by the radar due to the viewing geometry are flagged as shadow.\n\nCalibrated data is then orthorectified to the appropriate UTM projection. The orthorectified output maintains the original sample sizes (in range and azimuth) and was not shifted to any specific grid.\n\nRTC data is processed only for the Interferometric Wide Swath (IW) mode, which is the main acquisition mode over land and satisfies the majority of service requirements.\n",
        "extent": {
            "spatial": {"bbox": [overall_bbox]},
            "temporal": {"interval": [["2019-01-01T00:00:00Z", "2024-12-31T23:59:59Z"]]}
        },
        "license": "CC-BY-4.0",
        "keywords": ["sentinel-1", "rtc", "sar", "copernicus", "remote-sensing"],
        "providers": [{
            "name": "ESA",
            "roles": ["producer"],
            "url": "https://sentinel.esa.int"
        }],
        "links": [{
            "rel": "root",
            "href": f"{base_url}/collection.json",
            "type": "application/json"
        }] + [{
            "rel": "item",
            "href": f"{base_url}/{item['id']}/{item['id']}.json",
            "type": "application/json"
        } for item in items if item]
    }

    with open(output_dir / "collection.json", 'w') as f:
        json.dump(collection, f, indent=2)

    print("Created collection.json")

In [39]:
def main():
    base_url = "file:///G:/Semester4/Innolab/eoAPI/data/sentinel-1-rtc"
    indices_dir = Path(base_url.replace("file:///", ""))

    scene_dirs = [d for d in indices_dir.iterdir() if d.is_dir()]
    print(f"Found {len(scene_dirs)} scenes")

    items = []
    for scene_dir in scene_dirs:
        scene_id = scene_dir.name
        item = create_s1_rtc_item_json(scene_id, scene_dir, base_url)
        if item:
            items.append(item)
            item_file = scene_dir / f"{scene_id}.json"
            with open(item_file, 'w') as f:
                json.dump(item, f, indent=2)
            print(f"Created item for {scene_id}")

    create_s1_rtc_collection_json(indices_dir, items, base_url)

In [41]:
if __name__ == "__main__":
    main()

Found 13 scenes
Created item for S1A_IW_GRDH_1SDV_20240602T162124_20240602T162149_054146_0695A7_rtc
Created item for S1A_IW_GRDH_1SDV_20240604T034518_20240604T034543_054168_069665_rtc
Created item for S1A_IW_GRDH_1SDV_20240616T034517_20240616T034542_054343_069C72_rtc
Created item for S1A_IW_GRDH_1SDV_20240626T162123_20240626T162148_054496_06A1C8_rtc
Created item for S1A_IW_GRDH_1SDV_20240710T034516_20240710T034541_054693_06A89D_rtc
Created item for S1A_IW_GRDH_1SDV_20240720T162122_20240720T162147_054846_06ADEE_rtc
Created item for S1A_IW_GRDH_1SDV_20240722T034516_20240722T034541_054868_06AEAC_rtc
Created item for S1A_IW_GRDH_1SDV_20240801T162122_20240801T162147_055021_06B407_rtc
Created item for S1A_IW_GRDH_1SDV_20240803T034516_20240803T034541_055043_06B4B9_rtc
Created item for S1A_IW_GRDH_1SDV_20240813T162122_20240813T162147_055196_06BA45_rtc
Created item for S1A_IW_GRDH_1SDV_20240815T034515_20240815T034540_055218_06BB0E_rtc
Created item for S1A_IW_GRDH_1SDV_20240825T162123_20240825T1